## Environment Set-Up
Setting up a virtual environment is a crucial step in managing your Python project's dependencies effectively. 

###  1. Create a Virtual Environment
Run the following command to create a virtual environment in your project directory. 

Replace `.venv` with your desired name for the virtual environment folder if needed:

`python -m venv .venv`


###  2. Activate the Virtual Environment
Activate the virtual environment using the appropriate command for your operating system:
- On Linux/macOS:

    `source .venv/bin/activate`
    
- On Windows (Command Prompt)

    `.venv\Scripts\activate
`

- On Windows (PowerShell):

    `.venv\Scripts\Activate.ps1`

###  3. Upgrade `pip`
After activating the virtual environment, upgrade `pip` to ensure you have the latest version:

`pip install --upgrade pip`

###  4. Install Project Dependencies
Use `pip` to install all the required dependencies for the project as specified in the `requirements.txt` file:

`pip install -r requirements.txt`

## Create a project in Azure AI Foundry

1. Follow the guidelines in this article (https://learn.microsoft.com/en-us/azure/ai-studio/how-to/create-projects?tabs=ai-studio). If you don't have them already, make sure to create new AI services and AI Search resources.

   <Image src='docs/create_project_in_azure_ai_foundry.png'>


2. **Obtain the Connection String**  
   Copy the connection string from your AI Studio project. It should follow this format: `eastus.api.azureml.ms;12345678-abcd-1234-9fc6-62780b3d3e05;my-resource-group;my-project-name`

3. **Set Environment Variables**  
Ensure you set the required environment variables in `../.env` file.

## Create a TimeGEN endpoint

1. **Find TimeGEN model in Model Catalog**

   <Image src='docs/time_gen_1.png'>


2. **Create an endpoint**  
   
   <Image src='docs/time_gen_2.png'>

3. **Set Environment Variables**  
Save endpoint and key variables in `../.env` file.

In [1]:
# load environment variables from the .env file
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential


project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ["AIPROJECT_CONNECTION_STRING"]
)

connections = project_client.connections.list()
for connection in connections:
    print(connection)

In [ ]:
from azure.ai.projects.models import ConnectionType

connections = project_client.connections.list(
    connection_type=ConnectionType.AZURE_OPEN_AI,
)
for connection in connections:
    print(connection)

In [ ]:
aoai_client = project_client.inference.get_azure_openai_client(
    api_version="2024-06-01")

response = aoai_client.chat.completions.create(
    model=os.environ["CHAT_MODEL"],  # Model deployment name
    messages=[
        {
            "role": "user",
            "content": "How many feet are in a mile?",
        },
    ],
)

print(response.choices[0].message.content)

# List all agents

In [15]:
agent_list = project_client.agents.list_agents().data
for _agent in agent_list:
    print(_agent)

# Delete all agents

In [ ]:
agent_list = project_client.agents.list_agents().data
for _agent in agent_list:
    project_client.agents.delete_agent(_agent.id)
    print(f'agent {_agent.name} deleted')

# Create Agent

In [ ]:
from azure.ai.projects.models import CodeInterpreterTool


# Create an instance of the CodeInterpreterTool
code_interpreter = CodeInterpreterTool()


# The CodeInterpreterTool needs to be included in creation of the agent
agent = project_client.agents.create_agent(
    model=os.environ["CHAT_MODEL"],
    name="code_agent",
    instructions="You are helpful agent",
    tools=code_interpreter.definitions, # optional
    tool_resources=code_interpreter.resources, # optional
)

print(f"Created agent, agent ID: {agent.id}")

In [ ]:
# List all agents
agent_list = project_client.agents.list_agents().data
for _agent in agent_list:
    print(_agent) 
    # description 會描述得很清楚
    # metadata 可以額外添加字段例如創建者, group等等方便後續做管理

In [ ]:
# Create a thread to hold the conversation
# 可以獲得某個時間段內的對話紀錄 不用自己存
thread = project_client.agents.create_thread()
print(f"Created thread, thread ID: {thread.id}")

In [ ]:
# Create a message
user_message = "Hello"
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=user_message,
)
print(f"Created message, message ID: {message.id}, content: {message.content}")

In [ ]:
# Run the agent
run = project_client.agents.create_and_process_run(
    thread_id=thread.id, assistant_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

In [ ]:
# Display all messages in the thread
from IPython.display import Markdown, display
import helper

messages = project_client.agents.list_messages(thread_id=thread.id)

display(Markdown(helper.get_conversation_md(messages)))

In [ ]:
print(messages)

In [25]:
user_message = "Could you please create a bar chart for the operating profit using the following data and provide the file to me? Company A: $1.2 million, Company B: $2.5 million, Company C: $3.0 million, Company D: $1.8 million"

In [ ]:
from pathlib import Path


thread = project_client.agents.create_thread()
print(f"Created thread, thread ID: {thread.id}")

# Create a message
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=user_message,
)
print(f"Created message, message ID: {message.id}")

# Run the agent
run = project_client.agents.create_and_process_run(
    thread_id=thread.id, assistant_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    # Check if you got "Rate limit is exceeded.", then you want to get more quota
    print(f"Run failed: {run.last_error}")


messages = project_client.agents.list_messages(thread_id=thread.id)
display(Markdown(helper.get_conversation_md(messages)))

In [ ]:
from IPython.display import Image, display


# Generate an image file for the bar chart
for image_content in messages.image_contents:
    print(f"Image File ID: {image_content.image_file.file_id}")
    file_name = f"{image_content.image_file.file_id}_image_file.png"
    project_client.agents.save_file(
        file_id=image_content.image_file.file_id, file_name=file_name)
    print(f"Saved image file to: {Path.cwd() / file_name}")

# Print the file path(s) from the messages
for file_path_annotation in messages.file_path_annotations:
    print(f"File Paths:")
    print(f"Type: {file_path_annotation.type}")
    print(f"Text: {file_path_annotation.text}")
    print(f"File ID: {file_path_annotation.file_path.file_id}")
    print(f"Start Index: {file_path_annotation.start_index}")
    print(f"End Index: {file_path_annotation.end_index}")
    
    saved_file_name = Path(file_path_annotation.text).name
    project_client.agents.save_file(
        file_id=file_path_annotation.file_path.file_id, file_name=saved_file_name)
    display(Image(filename=saved_file_name))
